# Delta Table Staleness Monitor

## Overview
This notebook monitors Delta tables across your Databricks Unity Catalog environment to identify stale tables that may need attention. It uses `DESCRIBE HISTORY` to evaluate the freshness of tables and classifies them for action (e.g., warn or auto-delete) based on configurable thresholds and catalog-specific rules.

## ⚙️ Configuration Guide
This notebook is designed to be highly configurable. Before running, you will need to customize several sections to match your environment's specific needs. This guide points you to the key areas that require your attention.

**1. Skipping Specific Assets:**
You must define which catalogs, schemas, or tables should be ignored by the monitor. This is critical for excluding system tables, temporary schemas, or irrelevant data.

**2. Defining Staleness Thresholds:**
The core logic of this tool depends on rules that define when a table is considered "stale." You can set different time thresholds (in days) for different catalogs and schemas.

**3. Customizing Actions (Warn vs. Auto-Delete):**
Based on the staleness rules, the script takes actions like sending a warning or automatically deleting a table. You must configure which action applies to which tables.

**4. Setting Up Notifications:**
The notebook is pre-configured to send alerts to a Slack channel. You will need to add your `SLACK_WEBHOOK_URL` to databricks secrets.
  
    
      
## Core Functionality
- Iterates through all eligible tables using the Databricks SDK
- Extracts metadata from `DESCRIBE HISTORY`, such as creation date, last data update, and last modified by
- Applies staleness thresholds based on catalog and schema pre-defined rules
- Flags tables with no recent data changes
- Outputs Slack alerts grouped by catalog, summarizing stale tables
- Autodeletes stale tables, if configured to do so

## Data Sources and Output
| Name | Type | Source/Destination | Description |
|------|------|--------------------|-------------|
| `table_metadata` | Input | Databricks SDK | List of all catalog.schema.table entries |
| `history_df` | Input | `DESCRIBE HISTORY` | History logs per Delta table |
| `stale_df` | Output | In-memory DataFrame | Table metadata with staleness status and actions |
| `Slack Alerts` | Output | Slack channel | Grouped alerts per catalog with stale table summary |

## Recommended Actions
| Action | Description |
|--------|-------------|
| AUTO DELETE | Table is in the `developer` catalog and has not had recent data changes; eligible for automatic deletion |
| WARN | Table has exceeded the staleness threshold and should be reviewed for potential cleanup |
| NONE | Table is either fresh or does not meet the criteria for a warning or deletion |


In [0]:
!pip install loguru -q

In [0]:
from loguru import logger
import os
import pandas as pd
import re
from databricks.sdk import WorkspaceClient
from dateutil.relativedelta import relativedelta
import requests

In [0]:
NOW = pd.Timestamp.now()

### 1. Constants for Skipping
This section defines which assets (catalogs, schemas, table types, and tables matching a specific name pattern) should be **excluded** from the staleness check.

**ACTION REQUIRED:** Modify the Python sets below to fit your Databricks environment. For example, you may want to skip catalogs used for sandboxing, schemas containing only views, or tables created by specific automated processes.

- `SKIP_CATALOGS`: Add any catalogs you wish to ignore entirely. The `system` catalog is included by default.
- `SKIP_SCHEMAS`: Add any schemas you wish to ignore. `information_schema` is a common one to exclude.
- `SKIP_TABLE_TYPES`: By default, `VIEW` and `MATERIALIZED_VIEW` are skipped as they don't contain their own data. You can add other table types if needed.
- `SKIP_TABLE_PATTERN`: Use this for more complex rules. It allows you to skip tables in a specific catalog and schema that match a regular expression.
- `SKIP_SECURABLE_KINDS`: Skips catalogs based on their metastore type, such as a PostgreSQL-linked catalog.

In [0]:
SKIP_CATALOGS = {"system"}
SKIP_SCHEMAS = {"information_schema"}
SKIP_TABLE_TYPES = {"VIEW", "MATERIALIZED_VIEW"}
SKIP_TABLE_PATTERN = {
    ("forge", "restaurants_universe"): r".*_[a-z]{2}$"
}
SKIP_SECURABLE_KINDS = {"POSTGRES"}

### 2. Stale Threshold Rules
This function defines the business logic for what makes a table "stale." The rules are based on the number of days since a table's data was last changed.

**ACTION REQUIRED:** Modify the `thresholds` dictionary inside the `get_stale_threshold` function to define your organization's staleness policies.

The rules are applied with the following priority:
1.  A specific `(catalog, schema)` pair.
2.  A general `(catalog, None)` rule that applies to all schemas within that catalog.
3.  A default value if no other rule matches.


In [0]:
def get_stale_threshold(catalog, schema):
    """
    Determines the number of days after which a table is considered stale, based on catalog and schema.

    Rules:
    - developer.*         → 120 days (auto-delete)
    - frontiers.*         → 90 days (Slack warning)
    - web.*               → 45 days (Slack warning)
    - cargo.samples       → 90 days (Slack warning)
    - All other tables    → 45 days (Slack warning)

    Parameters:
        catalog_name (str): The catalog name (e.g., 'web', 'developer')
        schema_name (str): The schema name within the catalog (e.g., 'samples')

    Returns:
        int: Number of days to consider a table stale
    """
    default_threshold = 45
    thresholds = {
        ("developer", None): 120,
        ("frontiers", None): 90,
        ("web", None): 45,
        ("cargo", "samples"): 90
    }
    return thresholds.get((catalog, schema), thresholds.get((catalog, None), default_threshold))

### 3. Action Logic
This section controls which action is taken for a stale table. The configuration is based on simple sets, making it easy to define which catalogs are targeted for which action.

**ACTION REQUIRED:**
* **`AUTO_DELETE_CATALOGS`**: Add the names of any catalogs where stale tables should be automatically dropped. **Use extreme caution with this list.**

The logic in `get_action_for_table` uses this set to assign the final action. If stale, not skipped and not in `AUTO_DELETE_CATALOGS`, it will default to **WARN**.

In [0]:
AUTO_DELETE_CATALOGS = {"developer"}

def get_action_for_table(is_stale, table_info):
    """
    Determines the action to take based on table properties and staleness.

    Args:
        is_stale (bool): Whether the table has been flagged as stale.
        table_info (dict): A dictionary containing table metadata 
                           (e.g., {"catalog": "dev", "schema": "temp", ...}).

    Returns:
        str or None: The action to take ("AUTO DELETE", "WARN") or None.
    """
    if not is_stale:
        return None
        
    catalog = table_info.get("catalog")
    if catalog in AUTO_DELETE_CATALOGS:
        return "AUTO DELETE"
    
    # Default action for any other stale table
    return "WARN"

### 4. Slack Warning Config
This section sets the function that sends warnings for stale tables to a designated Slack channel.

**ACTION REQUIRED:**
1.  **Get a Slack Webhook URL**
2.  **Set Enviroment Variables:** Update the `SLACK_WEBHOOK_URL` secret variable for the compute.

In [0]:
SLACK_WEBHOOK_URL = os.getenv("SLACK_WEBHOOK_URL")
SLACK_CHANNEL = "slack-api-test"

def post_to_slack(payload: dict) -> bool:
    """Posts a payload to a Slack incoming webhook.

    This function sends a JSON payload to the `SLACK_WEBHOOK_URL`. It handles
    the request and logs errors if the message fails to send.

    Args:
        payload (dict): The message payload, typically containing a "blocks"
                        or "text" key, formatted for the Slack API.

    Returns:
        bool: True if the message was posted successfully (HTTP 200), False otherwise.
    """
    if not SLACK_WEBHOOK_URL:
        logger.error("SLACK_WEBHOOK_URL environment variable is not set. Cannot post message.")
        return False

    try:
        # Send the POST request to the Slack webhook URL
        response = requests.post(SLACK_WEBHOOK_URL, json=payload)

        # Raise an exception if the request returned an unsuccessful status code (4xx or 5xx)
        response.raise_for_status()

        return True
    except requests.exceptions.RequestException as e:
        # Log any network-related errors or unsuccessful status codes
        logger.error(f"Error posting message to Slack: {e}")
        return False
    except Exception as e:
        # Catch any other unexpected errors
        logger.error(f"An unexpected error occurred: {e}")
        return False

### Table Staleness Detection Logic

In [0]:
DATA_CHANGING_OPS = {
    "WRITE", "UPDATE", "DELETE", "INSERT", "MERGE", "COPY INTO",
    "RESTORE", "CREATE TABLE AS SELECT", "CREATE OR REPLACE TABLE AS SELECT",
    "REPLACE TABLE AS SELECT", "STREAMING UPDATE", "ROW TRACKING BACKFILL"
}

NON_DATA_CHANGING_OPS = {
    "CREATE", "ALTER", "DROP", "OPTIMIZE", "VACUUM START",
    "VACUUM END", "SET TBLPROPERTIES", "UPGRADE PROTOCOL"
}

In [0]:
def should_skip_table(catalog, schema, table):
    """
    Determines if a table should be skipped based on catalog, schema, type, or name pattern.

    Args:
        catalog: Catalog Object
        schema: Schema Object
        table: Table Object

    Returns:
        bool: True if the table should be excluded from processing
    """
    catalog_name = catalog.name
    schema_name = schema.name
    table_name = table.name
    table_type = table.table_type.value.upper() if table.table_type else None
    catalog_securable_kind = catalog.securable_kind.value.upper() if catalog.securable_kind else None

    if any(skip_kind in (catalog_securable_kind or '') for skip_kind in SKIP_SECURABLE_KINDS):
        logger.info(f"Skipping table: `{catalog_name}.{schema_name}.{table.name}` because the catalog has the securable kind:{catalog_securable_kind}.")
        return True
    if catalog_name in SKIP_CATALOGS:
        logger.info(f"Skipping table: `{catalog_name}.{schema_name}.{table.name}` because of the catalog.")
        return True
    if schema_name in SKIP_SCHEMAS:
        logger.info(f"Skipping table: `{catalog_name}.{schema_name}.{table.name}` because of the schema.")
        return True
    if table_type and table_type in SKIP_TABLE_TYPES:
        logger.info(f"Skipping table: `{catalog_name}.{schema_name}.{table.name}` because of the table type ({table_type}).")
        return True
    if (catalog_name, schema_name) in SKIP_TABLE_PATTERN:
        logger.info(f"Skipping table: `{catalog_name}.{schema_name}.{table.name}` because of the regex pattern in `{catalog_name}.{schema_name}` table.")
        pattern = SKIP_TABLE_PATTERN[(catalog_name, schema_name)]
        return re.match(pattern, table_name) is not None
    
    return False

In [0]:
def format_age(from_time, to_time=NOW):
    """
    Converts a timestamp difference into a human-readable string 
    like '2 months, 5 days' or '0 days' if recent.

    Parameters:
        from_time (pd.Timestamp): The earlier timestamp.
        to_time (pd.Timestamp): The later timestamp (default: current time).

    Returns:
        str: Human-readable duration between timestamps.
    """
    delta = relativedelta(to_time, from_time)
    parts = []
    if delta.years:
        parts.append(f"{delta.years} year{'s' if delta.years > 1 else ''}")
    if delta.months:
        parts.append(f"{delta.months} month{'s' if delta.months > 1 else ''}")
    if delta.days:
        parts.append(f"{delta.days} day{'s' if delta.days > 1 else ''}")
    return ", ".join(parts) if parts else "0 days"

In [0]:
def categorize_staleness(from_time, to_time=NOW):
    """
    Categorizes how old a timestamp is relative to a reference point, 
    returning a label like 'Years', 'Months', 'Days', or 'Recent'.

    Parameters:
        from_time (pd.Timestamp): The earlier timestamp.
        to_time (pd.Timestamp): The later timestamp (default: current time).

    Returns:
        str: A staleness category label.
              - 'Years' for 365+ days old
              - 'Months' for 30–364 days old
              - 'Days' for 1–29 days old
              - 'Recent' for 0 days
    """
    days_old = (to_time - from_time).days
    thresholds = {
        "Years": 365,
        "Months": 30,
        "Days": 1,
        "Recent": 0
    }
    
    # Evaluate thresholds in descending order of staleness
    for label, threshold in thresholds.items():
        if days_old >= threshold:
            return label
    return "Unknown"

In [0]:
# Initialize Databricks SDK workspace client
w = WorkspaceClient()

# This list will collect metadata about all eligible tables
table_metadata = []

# Loop through all catalogs returned by the SDK
for catalog in w.catalogs.list():
    catalog_name = catalog.name
    for schema in w.schemas.list(catalog_name):
        schema_name = schema.name
        for table in w.tables.list(catalog_name, schema_name):
            # Safely get the table type (e.g., MANAGED, VIEW)
            table_type = table.table_type.value if table.table_type else None

            # Apply custom logic to skip specific tables
            if should_skip_table(catalog, schema, table):
                continue

            # Store the metadata of tables that passed all filters
            table_metadata.append({
                "catalog": catalog_name,
                "schema": schema_name,
                "table": table.name,
                "full_path": f"{catalog_name}.{schema_name}.{table.name}",  # Used for DESCRIBE HISTORY
                "table_type": table_type,
                "comment": table.comment
            })

In [0]:
def parse_history_df(history_df, table_info):
    """
    Parses the output of DESCRIBE HISTORY for a Delta table to extract key metadata.

    This function analyzes the Delta table's operation history to:
    - Identify when the table was created and by whom.
    - Identify the most recent operation (e.g., INSERT, UPDATE, OPTIMIZE).
    - Identify the most recent *data-changing* operation (e.g., WRITE, DELETE, MERGE).
    - Calculate how long it has been since the last data change.
    - Determine if the table is considered stale based on catalog/schema-specific thresholds.
    - Return a dictionary of relevant fields for tracking table freshness and usage.

    Returns `None` if there are no data-changing operations, since such tables are not relevant for staleness tracking.
    
    Parameters:
    - history_df (pd.DataFrame): Output of `DESCRIBE HISTORY <table>` converted to a pandas DataFrame.

    Returns:
    - dict or None: Parsed metadata about the table, or None if no data-changing operations are found.
    """    
    # The last row in DESCRIBE HISTORY is the table's creation event
    created_row = history_df.iloc[-1]
    # The first row is the most recent operation (update, insert, etc.)
    last_updated_row = history_df.iloc[0]

    # Filter history to only include data-changing operations
    write_ops = history_df[history_df["operation"].isin(DATA_CHANGING_OPS)]
    # If present, pick the most recent data-changing operation
    last_data_row = write_ops.iloc[0] if not write_ops.empty else None

    # Extract creation timestamp and user
    created_at = created_row["timestamp"]
    created_by = created_row["userName"]

    # Extract last operation timestamp and user (regardless of whether it changed data)
    last_updated_at = last_updated_row["timestamp"]
    last_updated_by = last_updated_row["userName"]

    # Extract most recent data-changing operation info (if any)
    last_data_change = last_data_row["timestamp"] if last_data_row is not None else None
    last_data_changed_by = last_data_row["userName"] if last_data_row is not None else None

    # If no data-changing op exists, skip this table (maybe it's only been ALTER-ed or OPTIMIZE-ed)
    if last_data_change is None:
        logger.warning(f"No data-changing ops found in: {table_path}")
        return None

    # Calculate how many days it's been since the last data change
    days_outdated = (NOW - last_data_change).days

    # Determine if the table is considered stale based on catalog+schema rules
    is_stale = days_outdated > get_stale_threshold(table_info['catalog'], table_info['schema'])

    return {
        "table_path": table_path,
        "created_at": created_at.date(),
        "created_by": created_by,
        "last_updated_at": last_updated_at.date(),
        "last_updated_by": last_updated_by,
        "last_data_change": last_data_change.date(),
        "last_data_changed_by": last_data_changed_by,
        "days_outdated": days_outdated,
        "is_stale": is_stale,
        "outdated_for": format_age(last_data_change),
        "staleness_category": categorize_staleness(last_data_change),
        "action": get_action_for_table(is_stale, table_info),
        "comment": table_info["comment"]
    }

# Iterate over each valid table we've collected from the SDK
records = []
for table_info in table_metadata[:5]:
    table_path = table_info["full_path"]  # e.g., catalog.schema.table
    logger.info(f"Analyzing table: {table_path}")

    history_df = spark.sql(f"DESCRIBE HISTORY {table_path}").toPandas()
    
    result = parse_history_df(history_df, table_info)
    if not result:
        continue

    # Append all relevant information into the records list
    records.append(result)

stale_df = pd.DataFrame(records)

In [0]:
display(stale_df.sort_values(by="days_outdated", ascending=False))

### Take Actions Based on Staleness


In [0]:
# Filter relevant rows
delete_records = stale_df[
    (stale_df["is_stale"]) & (stale_df["action"] == "AUTO DELETE")
]

for table_path in delete_records["table_path"].values:
    spark.sql(f"DROP TABLE {table_path}")
    logger.info(f"Dropped table: `{table_path}`")

In [0]:
# Extract catalog, schema, and table from table_path
stale_df[["catalog", "schema", "table"]] = stale_df["table_path"].str.split(".", expand=True)

# Filter relevant rows
warn_records = stale_df[
    (stale_df["is_stale"]) & (stale_df["action"] == "WARN")
]

# Group by catalog
grouped = warn_records.groupby("catalog")

for catalog, group in grouped:
    # Header message
    header = f"*Catalog: `{catalog}`* — These tables appear stale :warning:\n"

    # Build markdown table inside a code block
    table_md = "```" + group[["schema", "table", "last_data_change"]].to_markdown(index=False) + "```"
    message_text = header + table_md

    # Slack block
    message_payload = {
        "blocks": [
            {
                "type": "section",
                "text": {
                    "type": "mrkdwn",
                    "text": message_text
                }
            }
        ]
    }

    # Send to Slack
    logger.info(f"Attempting to post message for catalog: {catalog}")
    if post_to_slack(message_payload):
        logger.info(f"Successfully posted message for catalog: {catalog}")
    else:
        logger.error(f"Failed to post message for catalog: {catalog}")